In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

device = "cuda" if torch.cuda.is_available() else "cpu"


def create_prompt_with_chat_format(messages, bos="<s>", eos="</s>", add_bos=True):
    formatted_text = ""
    for message in messages:
        if message["role"] == "system":
            formatted_text += "<|system|>\n" + message["content"] + "\n"
        elif message["role"] == "user":
            formatted_text += "<|user|>\n" + message["content"] + "\n"
        elif message["role"] == "assistant":
            formatted_text += "<|assistant|>\n" + message["content"].strip() + eos + "\n"
        else:
            raise ValueError(
                "Tulu chat template only supports 'system', 'user' and 'assistant' roles. Invalid role: {}.".format(
                    message["role"]
                )
            )
    formatted_text += "<|assistant|>\n"
    formatted_text = bos + formatted_text if add_bos else formatted_text
    return formatted_text


def inference(input_prompts, model, tokenizer):
    input_prompts = [
        create_prompt_with_chat_format([{"role": "user", "content": input_prompt}], add_bos=False)
        for input_prompt in input_prompts
    ]

    encodings = tokenizer(input_prompts, padding=True, return_tensors="pt")
    encodings = encodings.to(device)

    with torch.inference_mode():
        outputs = model.generate(encodings.input_ids, do_sample=False, max_new_tokens=250)

    output_texts = tokenizer.batch_decode(outputs.detach(), skip_special_tokens=True)

    input_prompts = [
        tokenizer.decode(tokenizer.encode(input_prompt), skip_special_tokens=True) for input_prompt in input_prompts
    ]
    output_texts = [output_text[len(input_prompt) :] for input_prompt, output_text in zip(input_prompts, output_texts)]
    return output_texts


model_name = "ai4bharat/Airavata"

tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side="left")
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16).to(device)

input_prompts = [
    "मैं अपने समय प्रबंधन कौशल को कैसे सुधार सकता हूँ? मुझे पांच बिंदु बताएं।",
    "मैं अपने समय प्रबंधन कौशल को कैसे सुधार सकता हूँ? मुझे पांच बिंदु बताएं और उनका वर्णन करें।",
]
outputs = inference(input_prompts, model, tokenizer)
print(outputs)


In [ ]:
import kagglehub
import os
import shutil
from pathlib import Path

def combine_datasets():
    # Download first dataset
    print("Downloading emmarex/plantdisease dataset...")
    path1 = kagglehub.dataset_download("emmarex/plantdisease")
    print(f"Path to first dataset: {path1}")

    # Download second dataset
    print("Downloading vipoooool/new-plant-diseases-dataset...")
    path2 = kagglehub.dataset_download("vipoooool/new-plant-diseases-dataset")
    print(f"Path to second dataset: {path2}")

    # Create a combined directory
    combined_path = Path("/content/combined_plant_disease_dataset")
    combined_path.mkdir(exist_ok=True)

    # Copy contents from both datasets to the combined directory
    print("Combining datasets...")
    for src_path in [path1, path2]:
        for item in os.listdir(src_path):
            src = os.path.join(src_path, item)
            dst = os.path.join(combined_path, item)

            # Handle duplicate filenames
            if os.path.exists(dst):
                base, ext = os.path.splitext(item)
                i = 1
                while os.path.exists(dst):
                    new_name = f"{base}_{i}{ext}"
                    dst = os.path.join(combined_path, new_name)
                    i += 1

            if os.path.isdir(src):
                shutil.copytree(src, dst)
            else:
                shutil.copy2(src, dst)

    print(f"Combined dataset created at: {combined_path}")
    return combined_path

# Run the combination function
combined_dataset_path = combine_datasets()

In [ ]:
# Import required libraries
import os
import shutil
from pathlib import Path
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import display, HTML

# ==============================================
# DATASET PREPARATION
# ==============================================

# Define paths (from your output)
dataset1_path = "/root/.cache/kagglehub/datasets/emmarex/plantdisease/versions/1"
dataset2_path = "/root/.cache/kagglehub/datasets/vipoooool/new-plant-diseases-dataset/versions/2"
combined_path = "/content/combined_plant_disease_dataset"

# Create combined directory if not exists
Path(combined_path).mkdir(exist_ok=True)

# ==============================================
# MODEL TRAINING PIPELINE (OPTIMIZED)
# ==============================================

def prepare_dataset(dataset_path):
    """Create optimized data generators with augmentation"""
    train_datagen = keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255,
        rotation_range=30,  # Reduced from 40 for faster training
        width_shift_range=0.15,  # Reduced from 0.2
        height_shift_range=0.15,  # Reduced from 0.2
        shear_range=0.15,  # Reduced from 0.2
        zoom_range=0.15,  # Reduced from 0.2
        horizontal_flip=True,
        fill_mode='reflect',  # Better than 'nearest' for natural images
        validation_split=0.2
    )

    train_gen = train_datagen.flow_from_directory(
        dataset_path,
        target_size=(256, 256),
        batch_size=32,
        class_mode='categorical',
        subset='training',
        shuffle=True,
        seed=42  # For reproducibility
    )

    val_gen = train_datagen.flow_from_directory(
        dataset_path,
        target_size=(256, 256),
        batch_size=32,
        class_mode='categorical',
        subset='validation',
        shuffle=False  # No need to shuffle validation data
    )

    return train_gen, val_gen

def build_model(num_classes):
    """Create optimized EfficientNet model"""
    base_model = keras.applications.EfficientNetB3(
        include_top=False,
        weights='imagenet',
        input_shape=(256, 256, 3),
        pooling='avg'  # Better than flatten for transfer learning
    )

    # Freeze base layers initially
    base_model.trainable = False

    inputs = keras.Input(shape=(256, 256, 3))
    x = base_model(inputs)
    x = keras.layers.Dense(512, activation='relu',
                          kernel_regularizer=keras.regularizers.l2(0.01))(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Dropout(0.5)(x)
    outputs = keras.layers.Dense(num_classes, activation='softmax')(x)

    model = keras.Model(inputs, outputs)

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.0001),
        loss='categorical_crossentropy',
        metrics=['accuracy',
                keras.metrics.Precision(name='precision'),
                keras.metrics.Recall(name='recall')]
    )

    return model, base_model

# ==============================================
# TRAINING EXECUTION (15 EPOCHS)
# ==============================================

# Prepare data generators
train_gen, val_gen = prepare_dataset(combined_path)
class_names = list(train_gen.class_indices.keys())

# Build model
model, base_model = build_model(len(class_names))

# Train for 15 epochs with callbacks
history = model.fit(
    train_gen,
    epochs=15,  # Using 15 epochs as requested
    validation_data=val_gen,
    callbacks=[
        keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=3,
            restore_best_weights=True
        ),
        keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.2,
            patience=2,
            min_lr=1e-6
        ),
        keras.callbacks.ModelCheckpoint(
            'best_model.h5',
            save_best_only=True,
            monitor='val_accuracy'
        )
    ],
    verbose=1
)

# Fine-tune for better performance (optional)
if input("Perform fine-tuning? (y/n): ").lower() == 'y':
    base_model.trainable = True
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=1e-5),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    history = model.fit(
        train_gen,
        epochs=5,  # Additional 5 epochs for fine-tuning
        initial_epoch=15,
        validation_data=val_gen,
        callbacks=[
            keras.callbacks.EarlyStopping(patience=2),
            keras.callbacks.ModelCheckpoint('fine_tuned_model.h5')
        ],
        verbose=1
    )

# ==============================================
# EVALUATION AND PREDICTION
# ==============================================

def evaluate_model(model, generator):
    """Enhanced model evaluation"""
    results = model.evaluate(generator)
    print(f"\nValidation Accuracy: {results[1]:.2%}")
    print(f"Precision: {results[2]:.2%}")
    print(f"Recall: {results[3]:.2%}")

# Evaluate the model
evaluate_model(model, val_gen)

# Rest of your prediction and reporting code remains the same...